In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import pandas as pd, numpy as np, tensorflow as tf

In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [3]:
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-25ms-24hr-28-2-24.feather')
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-w-landscape-full-25ms-24hr-28-2-24.feather')
df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather')
df = df[df.f < df.fg]
df['gust_factor'] = df.fg / df.f

df = df.drop(['f', 'fg', 'fsdev', 'd', 'dsdev', 'longitude', 'latitude', 'X', 'Y', 'time', 'stod'], axis = 1)# + [f'Landscape_{i}' for i in range(70)], axis = 1)

df_unfolded = df.elevations.apply(pd.Series)
df = pd.concat([df, df_unfolded], axis = 1)

df = df.dropna()
df = df.reset_index(drop = True)

In [4]:
df.iloc[:, -70:] = df.iloc[:, -70:].sub(df.station_elevation, axis = 0)

In [5]:
from sklearn.decomposition import PCA

df_landscape_elevation = df.iloc[:, -70:]

df_landscape_elevation = (df_landscape_elevation - df_landscape_elevation.mean()) / df_landscape_elevation.std()

n_components = 5

pca = PCA(n_components=n_components)
compressed_features = pca.fit_transform(df_landscape_elevation)

compressed_df = pd.DataFrame(data = compressed_features, columns = ['PC' + str(i) for i in range(n_components)])

df  = pd.concat([df, compressed_df], axis = 1)

In [6]:
y = df.gust_factor
X = df.drop(['gust_factor'], axis = 1)
X = X.drop(['ws_15', 'ws_250', 'ws_500', 'wd_15', 'wd_250', 'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500', 'N_01', 'N_12', 'landscape_points', 'elevations']
           + [i for i in range(70)], axis = 1)

# Changing the type of X,y so as to work with Tensorflow
X, y = X.values.astype(np.float32), y.values.astype(np.float32)
scaler = StandardScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [30]:
model = Sequential()
model.add(LSTM(25, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(25))
model.add(Dense(1))  # Output layer for regression


In [37]:
model.compile(optimizer='adam', loss = mean_absolute_percentage_error)
model.fit(X_train, y_train, epochs = 20, batch_size = 64, validation_data=(X_val, y_val))

Epoch 1/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 7.4899 - val_loss: 6.9818
Epoch 2/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 7.4565 - val_loss: 6.9369
Epoch 3/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 7.4307 - val_loss: 7.1069
Epoch 4/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 7.5432 - val_loss: 7.6374
Epoch 5/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 7.3660 - val_loss: 7.2028
Epoch 6/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 7.3424 - val_loss: 6.9620
Epoch 7/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 7.4423 - val_loss: 6.9151
Epoch 8/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 7.3786 - val_loss: 7.0767
Epoch 9/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 7.3306 - val_loss: 6.8717
Epoch 10/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 7.4846 - val_loss: 7.0153
Epoch 11/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 7.5042 - val_loss: 6.8342
Epoch 12/20
207/207 ━━━━━━━━━━━━━━━━━━━━ 

In [38]:
y_pred = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE: {mape} %")

115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
MAPE: 8.439703941345215 %


In [11]:
model.save("./saved_models/rnn.keras")